<a href="https://colab.research.google.com/github/bkkaggle/pytorch-CycleGAN-and-pix2pix/blob/master/pix2pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Dataset

In [ ]:
import os
import config
import datagan
import shutil

conf = config.Config()

In [ ]:
## Install

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
import os
print(f"current directory is : {os.path.basename(os.getcwd())}")
if os.path.basename(os.getcwd()) != "pytorch-CycleGAN-and-pix2pix":
    os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [ ]:
!pip install -r requirements.txt

In [ ]:
## Augment Data

In [ ]:
dataset = datagan.DataForGan()
dataset.augmente_data()

In [ ]:
print(conf.GAN_TRAIN_MASK,conf.GAN_COMBINED_DATASET,conf.GAN_TRAIN_MRI)
!python datasets/combine_A_and_B.py --fold_A {conf.GAN_TRAIN_MASK} --fold_B {conf.GAN_TRAIN_MRI} --fold_AB {conf.GAN_COMBINED_DATASET}

# Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_pix2pix_model.sh [edges2shoes, sat2map, map2sat, facades_label2photo, and day2night]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [ ]:
!bash ./scripts/download_pix2pix_model.sh facades_label2photo

# Training

-   `python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transfrom from class B to A.

In [ ]:
!python train.py --dataroot {conf.GAN_COMBINED_DATASET} --name {conf.GAN_MODEL_NAME} --model pix2pix --direction AtoB --num_threads 2 --batch_size 32

# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

In [ ]:
!ls checkpoints/

In [ ]:
!python test.py --dataroot ../data/gan_dataset/mask_mri --direction AtoB --model pix2pix --name {conf.GAN_MODEL_NAME}

# Generate Masks

In [ ]:
from src.random_shapes import generate
import os
if not os.path.isdir(conf.GAN_GENERATED_IMG):
    os.mkdir(conf.GAN_GENERATED_IMG)

if not os.path.isdir(conf.GAN_GENERATED_MASKS):
    os.mkdir(conf.GAN_GENERATED_MASKS)

if not os.path.isdir(conf.GAN_GENERATED_MRI):
    os.mkdir(conf.GAN_GENERATED_MRI)

generate(number_of_masks_to_generate=33000, start_index=20212, path=conf.GAN_GENERATED_MASKS)

# Generate MRI images

In [ ]:
os.chdir(os.path.join(conf.WORKING_DIR, "pytorch-CycleGAN-and-pix2pix"))
!python test.py --dataroot {conf.GAN_GENERATED_MASKS} --name {conf.GAN_MODEL_NAME} --model test --netG unet_256 --direction AtoB --dataset_mode single --norm batch --results_dir {conf.GAN_GENERATED_MRI} --max_dataset_size 33000 --num_test 33000

Move generated images to data directory

In [ ]:
import shutil
import os

os.chdir(os.path.join(conf.DATASET_DIR, conf.GAN_MODEL_NAME, "test_latest", "images"))
for file_name in os.listdir():
    if file_name.find("fake")!= -1:
        target_file_name = os.path.join(conf.GAN_GENERATED_MRI, file_name.replace("_fake", ""))
        shutil.move(file_name, target_file_name)